In [1]:
import psycopg2
import json
import codecs
from ast import literal_eval

<h1> connect to db and create table </h1>

In [2]:
conn = psycopg2.connect("host=localhost port=5432 dbname=amazon user=postgres password=darkmatter")
cur = conn.cursor()

<h1> Create a Table with json as the column </h1> 

In [3]:
#Metadata db
cur.execute("CREATE TABLE metadata (id serial PRIMARY KEY,\
    metajson json,\
    asin text,\
    title text\
    );")
conn.commit() #submit change to db

<h1> read in 10 GB file one line at a time and write to the db </h1>

In [ ]:
with open('/home/ubuntu/amazon/metadata.json') as f:
    for line in f:
        metajson=json.loads(json.dumps(literal_eval(line)))
        try:
            asin=metajson['asin']
        except KeyError:
            asin=''
        try:
            title=metajson['title']
        except KeyError:
            title=''
        
        #insert into postgres
        cur.execute("INSERT INTO metadata (\
        metajson,\
        asin,\
        title\
        )\
        VALUES (\
        %s,%s,%s\
        )",(json.dumps(metajson),asin,title)\
        )
        conn.commit() #submit change to db

In [11]:
metajson

{'asin': '0001048791',
 'categories': [['Books']],
 'imUrl': 'http://ecx.images-amazon.com/images/I/51MKP0T4DBL.jpg',
 'salesRank': {'Books': 6334800},
 'title': 'The Crucible: Performed by Stuart Pankin, Jerome Dempsey &amp; Cast'}

<h1> create the demo meta data table to speed up query </h1>

In [7]:
#QA db #metadata_cell_phones_and_accessories
cur.execute("CREATE TABLE metadata_demo (id serial PRIMARY KEY,\
    metajson json,\
    asin text,\
    title text\
    );")
conn.commit() #submit change to db

In [3]:
#demo_asins=["B0009H2M1Y","B0002WRGHG","B00002X29G","8288878881"]
# the 5 most similar asins for each asin, determined by doc2vec similarity on reviews
#demo_asins=['B007THM9R4', 'B005LFXBLE', 'B003FWLZTK', 'B001LRPN0S', 'B000S5Q9CA']+\
#['B0007WWAGI', 'B0006GFARG', 'B0009B0IX4', 'B000SAD7RA', 'B00069IQG0']+\
#['B00822XBGI', 'B004SK3LSG', 'B0046UQK36', 'B003ZU5X36', 'B00C1BKF3I']+\
#['B004EYSKM8', 'B009YDFX7Y', 'B000S5Q9CA', 'B006YCORR0', 'B00B61ZZB6']

#with key words
demo_asins=['B00822XBGI', 'B002A9PVAI', 'B007Y8TP52', 'B00792JJK4', 'B004L08BAG']+\
['B004PCJQGS', 'B009SQSV7G', 'B001XXUOQI', 'B009YDFX7Y', 'B004T0GHOU']

for asin in demo_asins:
    cur.execute("SELECT metajson,title from metadata WHERE asin= %s ;",(asin,))
    result=cur.fetchall()[0]
    metajson=result[0]
    title=result[1]
    
    #insert into postgres
    cur.execute("INSERT INTO metadata_demo (\
    metajson,\
    asin,\
    title\
    )\
    VALUES (\
    %s,%s,%s\
    )",(json.dumps(metajson),asin,title)\
    )
    conn.commit() #submit change to db

In [4]:
conn.close()

<h1> Select asin, title and metadata from metadata for asin in reviews_cell_phones_and_accessories </h1>

In [3]:
cur.execute("SELECT a.asin, b.title, b.metajson FROM reviews_cell_phones_and_accessories a \
left join metadata b on a.asin=b.asin;")
results=cur.fetchall()

<h3> insert into table </h3>

In [8]:
for result in results:
    asin=result[0]
    title=result[1]
    metajson=result[2]
    
    #insert into postgres
    cur.execute("INSERT INTO metadata_cell_phones_and_accessories (\
    metajson,\
    asin,\
    title\
    )\
    VALUES (\
    %s,%s,%s\
    )",(json.dumps(metajson),asin,title)\
    )
    conn.commit() #submit change to db

In [9]:
conn.close()